In [ ]:
import os
import io
import sys
import tensorflow as tf
import pandas as pd
from PIL import Image
path = '/home/chrisjan/project/models/object_detection'
os.chdir(path)
sys.path.append('..')
from object_detection.utils import dataset_util

raccoon_image_dir = '/home/chrisjan/project/training/raccoon/images'
train_csv_file = '/home/chrisjan/project/training/raccoon/data/train_labels.csv'
test_csv_file = '/home/chrisjan/project/training/raccoon/data/test_labels.csv'
output_train_record_file = '/home/chrisjan/project/training/raccoon/data/train.record'
output_test_record_file = '/home/chrisjan/project/training/raccoon/data/test.record'

In [ ]:
def create_tf_example(row):
    full_path = raccoon_image_dir + '/' + row['filename']
    with tf.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    filename = row['filename'].encode('utf8')
    image_format = b'jpg'
    xmins = [row['xmin'] / width]
    xmaxs = [row['xmax'] / width]
    ymins = [row['ymin'] / height]
    ymaxs = [row['ymax'] / height]
    classes_text = [row['class'].encode('utf8')]
    classes = [1]
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [ ]:
train_examples = pd.read_csv(train_csv_file)
writer_train = tf.python_io.TFRecordWriter(output_train_record_file)
for index, row in train_examples.iterrows():
    train_example = create_tf_example(row)
    writer_train.write(train_example.SerializeToString())
writer_train.close()
test_examples = pd.read_csv(test_csv_file)
writer_test = tf.python_io.TFRecordWriter(output_test_record_file)
for index, row in test_examples.iterrows():
    test_example = create_tf_example(row)
    writer_test.write(test_example.SerializeToString())
writer_test.close()